# Apply ARN Categories

In [1]:
from pathlib import Path

In [2]:
TOP = Path.cwd().as_posix().replace('notebooks','')

In [3]:
TOP

'/home/grace/Documents/python/arn_cats/'

In [4]:
raw_dir = Path(TOP) / 'data' /'raw'
model_dir = Path(TOP) / 'arn_cats' /'data'

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import textwrap

In [6]:
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path
sys.path.insert(0, project_root)


In [7]:
from arn_cats.utl import logger
log = logger.get_logger(__name__)

In [8]:
from rdkit import Chem
import pickle
import numpy as np
import pandas as pd
from glob import glob
import textwrap

In [9]:
from arn_cats.chm import cheminfo_toolkit

No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float
No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float
No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float
No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float
No registered converter was able to produce a C++ rvalue of type std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float
No registered converter was able to produce a C++ rvalue of type std::basic_string<wc

In [13]:
from arn_cats.chm.cheminfo_toolkit import Molecule, Fingerprint_engine

In [10]:
from arn_cats.data.data_load import arn_groups

In [11]:
arn_groups

,entry ID,Group_number,Group_name_ARN,Substance_name_ARN,EC_number_ARN,CAS_number_ARN,DSSTox_ID,DSSTox_structure_ID,DSSTox_QC_Level,Substance_name_DSSTox,CAS_number_DSSTox,Substance_type_DSSTox,Substance_Note_DSSTox,SMILES_DSSTox,InChI_DSSTox,InChIKey_DSSTox,Formula_DSSTox,MolWeight_DSSTox,SMILES_2D_QSAR_DSSTox
0,0,0,triphenylphosphite and its derivatives,"Tris(2,4-ditert-butylphenyl) phosphite",250-709-6,31570-04-4,DTXSID2027969,DTXCID907969,DSSTox_High,"Tris(2,4-di-tert-butylphenyl) phosphite",31570-04-4,Single Compound,NaN,CC(C)(C)C1=CC(=C(OP(OC2=CC=C(C=C2C(C)(C)C)C(C)...,"InChI=1S/C42H63O3P/c1-37(2,3)28-19-22-34(31(25...",JKIJEFPNVSHHEI-UHFFFAOYSA-N,C42H63O3P,646.937,CC(C)(C)C1=CC(=C(OP(OC2=CC=C(C=C2C(C)(C)C)C(C)...
1,1,0,triphenylphosphite and its derivatives,Triphenyl phosphite,202-908-4,101-02-0,DTXSID0026252,DTXCID306252,DSSTox_High,Triphenyl phosphite,101-02-0,Single Compound,NaN,O(P(OC1=CC=CC=C1)OC1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C18H15O3P/c1-4-10-16(11-5-1)19-22(20-...,HVLLSGMXQDNUAL-UHFFFAOYSA-N,C18H15O3P,310.289,O(P(OC1=CC=CC=C1)OC1=CC=CC=C1)C1=CC=CC=C1
2,2,0,triphenylphosphite and its derivatives,"6,6'-[(3,3'-di-tert-butyl-5,5'-dimethoxybiphen...",700-178-6,121627-17-6,DTXSID60746295,DTXCID301767468,DSSTox_High,"6,6′-[[3,3′-Bis(1,1-dimethylethyl)-5,5′-dimeth...",121627-17-6,Single Compound,NaN,COC1=CC(=C(OP2OC3=CC=CC=C3C3=CC=CC=C3O2)C(=C1)...,"InChI=1S/C46H44O8P2/c1-45(2,3)37-27-29(47-7)25...",WUFGFUAXCBPGOL-UHFFFAOYSA-N,C46H44O8P2,786.798,NaN
3,3,0,triphenylphosphite and its derivatives,"5,7-Di-t-butyl-3-[3,5-dimethyl-4-[(1,3,7,9-tet...",817-187-7,1803088-15-4,DTXSID101019508,DTXCID901777736,DSSTox_High,"5,7-Di-tert-butyl-3-{3,5-dimethyl-4-[(2,4,8,10...",1803088-15-4,Mixture of Stereoisomers,NaN,CC1C2=C(OP(OC3=C(C)C=C(C=C3C)C3C(=O)OC4=C3C=C(...,InChI=1/C54H73O5P/c1-30-22-33(43-39-26-36(51(1...,LXQBVTPPBNWKGG-UHFFFAOYNA-N,C54H73O5P,833.147,NaN
4,4,0,triphenylphosphite and its derivatives,Tris(nonylphenyl) phosphite,247-759-6,26523-78-4,DTXSID8027903,NaN,DSSTox_High,Nonylphenolphosphite(3:1),26523-78-4,Mixture of Stereoisomers,Ill-defined substance,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,2179,85,(tetrahydro)furan primary alcohol derivatives ...,"2S-(2-furyl)-5R-hydroxy-4R-(1R,2-dihydroxy)eth...",422-440-6,7089-59-0,DTXSID90991175,DTXCID101768646,DSSTox_High,"2,4-Monofurfurylidenesorbitol",7089-59-0,Mixture of Stereoisomers,NaN,[H][C@@]1(OC(O[C@@H](CO)[C@H]1O)C1=CC=CO1)[C@H...,InChI=1S/C11H16O7/c12-4-6(14)10-9(15)8(5-13)17...,YZNXOCVTNIERFC-NYONGVCJSA-N,C11H16O7,260.242,NaN
2180,2180,85,(tetrahydro)furan primary alcohol derivatives ...,2-[(tetrahydrofurfuryl)oxy]ethanol,227-407-8,5831-59-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2181,2181,85,(tetrahydro)furan primary alcohol derivatives ...,"2,5-anhydro-3,4-dideoxyhexitol",203-239-0,104-80-3,DTXSID40870446,DTXCID60818168,DSSTox_High,"2,5-Tetrahydrofurandimethanol",104-80-3,Mixture of Stereoisomers,NaN,OCC1CCC(CO)O1,"InChI=1/C6H12O3/c7-3-5-1-2-6(4-8)9-5/h5-8H,1-4...",YCZZQSFWHFBKMU-UHFFFAOYNA-N,C6H12O3,132.159,OCC1CCC(CO)O1
2182,2182,85,(tetrahydro)furan primary alcohol derivatives ...,(S)-(-)-tetrahydro-2-furoic acid,433-770-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:

# read in molecules and put them in a list
molecules = []
n_succeeded = 0
n_failed = 0
fingerprint_engine = Fingerprint_engine.Morgan(radius=2, nBits=2560)
for idx, row in arn_groups.iterrows():
    smiles = row['SMILES_DSSTox']
    try:
        mol = Molecule.from_smiles(smiles)
        mol.group = row['Group_name_ARN']
        mol.group_number = row['Group_number']
        mol.CASRN = row['CAS_number_ARN']
        mol.compute_fingerprint(fingerprint_engine)
        molecules.append(mol) 
        arn_groups.loc[idx, 'structure processable'] = True
    except Exception as e:
        log.error(e)
        n_failed += 1
        arn_groups.loc[idx, 'structure processable'] = False
arn_groups['structure processable'] = arn_groups['structure processable'].astype(bool)
arn_groups.to_excel(model_dir / 'ARN_groups.xlsx')
with open(model_dir/'molecules_all.pickle', 'wb') as handle:
    pickle.dump(molecules, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [15]:
arn_groups

,entry ID,Group_number,Group_name_ARN,Substance_name_ARN,EC_number_ARN,CAS_number_ARN,DSSTox_ID,DSSTox_structure_ID,DSSTox_QC_Level,Substance_name_DSSTox,CAS_number_DSSTox,Substance_type_DSSTox,Substance_Note_DSSTox,SMILES_DSSTox,InChI_DSSTox,InChIKey_DSSTox,Formula_DSSTox,MolWeight_DSSTox,SMILES_2D_QSAR_DSSTox,structure processable
0,0,0,triphenylphosphite and its derivatives,"Tris(2,4-ditert-butylphenyl) phosphite",250-709-6,31570-04-4,DTXSID2027969,DTXCID907969,DSSTox_High,"Tris(2,4-di-tert-butylphenyl) phosphite",31570-04-4,Single Compound,NaN,CC(C)(C)C1=CC(=C(OP(OC2=CC=C(C=C2C(C)(C)C)C(C)...,"InChI=1S/C42H63O3P/c1-37(2,3)28-19-22-34(31(25...",JKIJEFPNVSHHEI-UHFFFAOYSA-N,C42H63O3P,646.937,CC(C)(C)C1=CC(=C(OP(OC2=CC=C(C=C2C(C)(C)C)C(C)...,True
1,1,0,triphenylphosphite and its derivatives,Triphenyl phosphite,202-908-4,101-02-0,DTXSID0026252,DTXCID306252,DSSTox_High,Triphenyl phosphite,101-02-0,Single Compound,NaN,O(P(OC1=CC=CC=C1)OC1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C18H15O3P/c1-4-10-16(11-5-1)19-22(20-...,HVLLSGMXQDNUAL-UHFFFAOYSA-N,C18H15O3P,310.289,O(P(OC1=CC=CC=C1)OC1=CC=CC=C1)C1=CC=CC=C1,True
2,2,0,triphenylphosphite and its derivatives,"6,6'-[(3,3'-di-tert-butyl-5,5'-dimethoxybiphen...",700-178-6,121627-17-6,DTXSID60746295,DTXCID301767468,DSSTox_High,"6,6′-[[3,3′-Bis(1,1-dimethylethyl)-5,5′-dimeth...",121627-17-6,Single Compound,NaN,COC1=CC(=C(OP2OC3=CC=CC=C3C3=CC=CC=C3O2)C(=C1)...,"InChI=1S/C46H44O8P2/c1-45(2,3)37-27-29(47-7)25...",WUFGFUAXCBPGOL-UHFFFAOYSA-N,C46H44O8P2,786.798,NaN,True
3,3,0,triphenylphosphite and its derivatives,"5,7-Di-t-butyl-3-[3,5-dimethyl-4-[(1,3,7,9-tet...",817-187-7,1803088-15-4,DTXSID101019508,DTXCID901777736,DSSTox_High,"5,7-Di-tert-butyl-3-{3,5-dimethyl-4-[(2,4,8,10...",1803088-15-4,Mixture of Stereoisomers,NaN,CC1C2=C(OP(OC3=C(C)C=C(C=C3C)C3C(=O)OC4=C3C=C(...,InChI=1/C54H73O5P/c1-30-22-33(43-39-26-36(51(1...,LXQBVTPPBNWKGG-UHFFFAOYNA-N,C54H73O5P,833.147,NaN,True
4,4,0,triphenylphosphite and its derivatives,Tris(nonylphenyl) phosphite,247-759-6,26523-78-4,DTXSID8027903,NaN,DSSTox_High,Nonylphenolphosphite(3:1),26523-78-4,Mixture of Stereoisomers,Ill-defined substance,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,2179,85,(tetrahydro)furan primary alcohol derivatives ...,"2S-(2-furyl)-5R-hydroxy-4R-(1R,2-dihydroxy)eth...",422-440-6,7089-59-0,DTXSID90991175,DTXCID101768646,DSSTox_High,"2,4-Monofurfurylidenesorbitol",7089-59-0,Mixture of Stereoisomers,NaN,[H][C@@]1(OC(O[C@@H](CO)[C@H]1O)C1=CC=CO1)[C@H...,InChI=1S/C11H16O7/c12-4-6(14)10-9(15)8(5-13)17...,YZNXOCVTNIERFC-NYONGVCJSA-N,C11H16O7,260.242,NaN,True
2180,2180,85,(tetrahydro)furan primary alcohol derivatives ...,2-[(tetrahydrofurfuryl)oxy]ethanol,227-407-8,5831-59-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2181,2181,85,(tetrahydro)furan primary alcohol derivatives ...,"2,5-anhydro-3,4-dideoxyhexitol",203-239-0,104-80-3,DTXSID40870446,DTXCID60818168,DSSTox_High,"2,5-Tetrahydrofurandimethanol",104-80-3,Mixture of Stereoisomers,NaN,OCC1CCC(CO)O1,"InChI=1/C6H12O3/c7-3-5-1-2-6(4-8)9-5/h5-8H,1-4...",YCZZQSFWHFBKMU-UHFFFAOYNA-N,C6H12O3,132.159,OCC1CCC(CO)O1,True
2182,2182,85,(tetrahydro)furan primary alcohol derivatives ...,(S)-(-)-tetrahydro-2-furoic acid,433-770-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [16]:
arn_stats = (arn_groups
             .groupby(['Group_number', 'Group_name_ARN'])
             .agg(**{
                      "number of substances": pd.NamedAgg(column="entry ID", aggfunc=pd.Series.nunique),
                      "found in CompTox": pd.NamedAgg(column="DSSTox_ID", aggfunc=lambda s: len(s.dropna())),
                      "substance type": pd.NamedAgg(column="Substance_type_DSSTox", aggfunc=lambda s: s.value_counts().to_dict()),
                      "DSSTox QC level": pd.NamedAgg(column="DSSTox_QC_Level", aggfunc=lambda s: s.value_counts().to_dict()),
                      "SMILES available": pd.NamedAgg(column="SMILES_DSSTox", aggfunc=lambda s: len(s.dropna())),
                      "SMILES (2D-QSAR) available": pd.NamedAgg(column="SMILES_2D_QSAR_DSSTox", aggfunc=lambda s: len(s.dropna())),
                      "structure processable": pd.NamedAgg(column="structure processable", aggfunc=lambda s: s.sum()),
                })
             )

In [17]:
arn_stats

,,number of substances,found in CompTox,substance type,DSSTox QC level,SMILES available,SMILES (2D-QSAR) available,structure processable
Group_number,Group_name_ARN,,,,,,,
0,triphenylphosphite and its derivatives,20,20,"{'Mixture/Formulation': 13, 'Single Compound':...","{'DSSTox_Low': 10, 'DSSTox_High': 8, 'Public_H...",6,4,6
1,thioxanthenones,10,10,"{'Single Compound': 9, 'Mixture of Stereoisome...","{'Public_High_CAS': 6, 'DSSTox_High': 3, 'Publ...",10,8,10
2,thio alkyl acids and esters of dialkyldithiophosphates (DDP S acids and esters),9,9,"{'Single Compound': 5, 'Mixture/Formulation': ...","{'Public_High_CAS': 5, 'DSSTox_Low': 2, 'DSSTo...",6,6,6
3,tetrahydroxymethyl and tetraalkyl phosphonium salts,15,15,"{'Single Compound': 12, 'Mixture/Formulation': 3}","{'DSSTox_High': 9, 'Public_High_CAS': 3, 'Publ...",12,10,12
4,succinic anhydrides,23,23,"{'Mixture/Formulation': 15, 'Mixture of Stereo...","{'DSSTox_High': 11, 'Public_High': 5, 'DSSTox_...",6,6,6
...,...,...,...,...,...,...,...,...
81,Acyl glycinates and sarcosinates,30,30,"{'Single Compound': 18, 'Mixture/Formulation':...","{'DSSTox_High': 10, 'DSSTox_Low': 10, 'Public_...",19,17,19
82,"Acyl derivatives from alpha-amino acids other than glutamic acid, glycine or sarcosine",32,28,"{'Mixture/Formulation': 14, 'Single Compound':...","{'DSSTox_Low': 11, 'DSSTox_High': 8, 'Public_H...",14,10,14
83,2-hydroxybenzenecarbonyl oxime derivatives,7,7,"{'Mixture/Formulation': 4, 'Single Compound': 3}","{'DSSTox_Low': 4, 'Public_High': 2, 'Public_Hi...",3,2,3
